In [3]:
import yfinance as yf
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

from AA import DataDownloader, AssetAllocation

In [6]:
downloader = DataDownloader()

assets = ['AAPL', 'IBM', 'TSLA', 'GOOG', 'NVDA'] 
benchmark = '^GSPC'  
start_date = '2019-01-01'
end_date = '2023-12-31'
rf = .065

asset_prices, benchmark_prices = downloader.download_data(start_date= start_date, end_date= end_date,
                                                          assets= assets, benchmark=benchmark)


# Default limits for every asset (min 1% - max 100%):  boounds = tuple((0.01, 1) for _ in range(self.num_assets))
Asset_allocation = AssetAllocation(asset_prices = asset_prices,benchmark_prices = benchmark_prices, rf=rf)

[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  1 of 1 completed


In [7]:
Asset_allocation.Optimize_Portfolio(n_simulations = 10000) #Default Method == "Montecarlo"

,AAPL,GOOG,IBM,NVDA,TSLA,Optimized Value
Max Sharpe,0.176222,0.032408,0.015865,0.535341,0.240165,0.079650
Max Omega,0.067144,0.011577,0.039909,0.830420,0.050949,1.264147
Min VaR (Empirical),0.028690,0.027527,0.041368,0.089152,0.813263,-0.052241
Min VaR (Parametric),0.203531,0.181285,0.573807,0.020320,0.021057,-0.024602


In [8]:
Asset_allocation.Optimize_Portfolio(method = "SLSQP")

,AAPL,GOOG,IBM,NVDA,TSLA,Optimized Value
Max Sharpe,0.221457,0.010000,0.01000,0.50168,0.256863,0.079996
Max Omega,0.010000,0.010000,0.01000,0.96000,0.010000,1.288639
Min VaR (Empirical),0.010000,0.010000,0.01000,0.38920,0.580800,-0.049680
Min VaR (Parametric),0.156482,0.217159,0.60636,0.01000,0.010000,-0.024424


In [ ]:
Asset_allocation.Optimize_Portfolio(method = "Genetic", population_size=100, generations=200, crossover_rate=0.7, mutation_rate=0.1)

In [ ]:
Asset_allocation.Optimize_Portfolio(method = "Gradient", learning_rate=0.01, max_iters=1000, tol=1e-6)

-----

-------

--------------

----------